In [8]:
import os
import os.path as path
import re
import cv2

from model import airsim as model
from util import quickstart as util

## Parameters Setting

In [14]:
# 欲載入 pretrain weight 的路徑
MODEL_WEIGHT_PATH = '/home/mjchiu/Documents/darknet-depth/models/mnv2_segment_depth_multiloss/airsim/6class_epoch-20_loss-8.3198.h5'

# 資料路徑
DATA_PATH = '/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/data/2019-10-08-19-17'
RESULT_PATH = '/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# Video Output 解析度設定
OUTPUT_HEIGHT = 1440
OUTPUT_WIDTH = 1080

## Build Model

In [10]:
# 建立 Model layer 架構
model = model.MNv2_segment_depth_multiloss_model(
    inputShape = (INPUT_HEIGHT, INPUT_WIDTH, 3), 
    alpha = 0.35, 
    expansion_factor = 6, 
    depth_multiplier = 1, 
    lock_backend_weights = False,
    CLASSES = 6,
)

# 載入Model weight
model.load_weights(MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet_v2.py:315: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


## Predict the Image and Save Result

In [15]:
# iterate each dataset
for folder_name in os.listdir(DATA_PATH):
    folder_path = os.path.join(DATA_PATH, folder_name)    
    # prepare result image folder
    result_image_folder_path = os.path.join(RESULT_PATH, 'image', folder_name)
    if not os.path.exists(result_image_folder_path):
        os.makedirs(result_image_folder_path)
    for file_name in os.listdir(folder_path):
        # 讀取影像
        image = util.read_image(os.path.join(folder_path, file_name), shape=(INPUT_HEIGHT,INPUT_WIDTH), rand=0)
        # 使用 image 來預測出 depth map 和 semantic segmentation
        predict_depth, predict_segment = util.getResult(model, image, (INPUT_HEIGHT, INPUT_WIDTH))
        # 儲存結果
        image_path = os.path.join(RESULT_PATH, 'image', folder_name, file_name)
        util.saveResult([image, predict_depth[0], predict_segment[0]], image_path)
            

# Convert Images into Video 

In [17]:
# prepare result video folder
result_video_folder_path = path.join(RESULT_PATH, 'video')
if not os.path.exists(result_video_folder_path):
    os.makedirs(result_video_folder_path)
# iterate result images
result_image_folder =  path.join(RESULT_PATH, 'image')
for folder_name in os.listdir(result_image_folder):
    # create result video
    video_path = os.path.join(result_video_folder_path, folder_name+'.mp4')
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'MP4V'), 10, (OUTPUT_HEIGHT, OUTPUT_WIDTH))
    for file_name in sorted(os.listdir(folder_path), key=lambda x: int(re.split('\\-|\\.', x)[-2])):
        # 儲存結果
        image_path = os.path.join(result_image_folder, folder_name, file_name)
        #image = cv2.resize(cv2.imread(image_path), (OUTPUT_HEIGHT, OUTPUT_WIDTH))
        image = cv2.imread(image_path)
        video.write(image)
    video.release()
    print('{} created'.format(video_path))

/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17/video/2019-10-08-19-20-16.mp4 created
/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17/video/2019-10-08-19-18-55.mp4 created
/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17/video/2019-10-08-19-17-59.mp4 created
/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17/video/2019-10-08-19-17-31.mp4 created
/home/mjchiu/Documents/drone/collision_prevention/depth_prediction/result/2019-10-08-19-17/video/2019-10-08-19-18-22.mp4 created
